In [28]:
# !pip install -q langchain==0.1.9
# !pip install -q openai==1.13.3

In [29]:
from keys import openapi_key
import os

os.environ['OPENAI_API_KEY'] = openapi_key


In [30]:
from langchain_community.llms import OpenAI
llm = OpenAI(temperature=0.6)
name = llm.invoke("I want to open a restaurant for Chinese food. Suggest a fancy name for this.")
print(name)



"Dragon's Den Cuisine"


In [31]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ["cuisine"],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.",
)

prompt_template_name.format(cuisine="Mexican")

'I want to open a restaurant for Mexican food. Suggest a fancy name for this.'

In [35]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)
name_chain.invoke({"cuisine":"Mexican"})

{'cuisine': 'Mexican',
 'text': '\n\n"El Jardín de Sabores" (The Garden of Flavors)'}

## Sequential Chains

In [9]:
prompt_template_items = PromptTemplate(
    input_variables = ["restaurant_name"],
    template="Suggest some menu items for {restaurant_name}. Return it as a comma separated list.",
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [10]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain, food_items_chain])
response = chain.run("Indian")
print(response)



1. Chicken Tikka Masala
2. Vegetable Samosas
3. Lamb Vindaloo
4. Palak Paneer
5. Tandoori Chicken
6. Garlic Naan
7. Chana Masala
8. Aloo Gobi
9. Butter Chicken
10. Mango Lassi 



## Sequential Chain

In [11]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables=["cuisine"],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this.",
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template = "Suggest some menu items for {restaurant_name}. Return the menu items as a single, comma separated string with no additional preamble.",
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")



In [13]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[name_chain, food_items_chain], 
    input_variables=["cuisine"], 
    output_variables = ["restaurant_name", "menu_items"]
)

chain.invoke({"cuisine": "Singaporean"})

{'cuisine': 'Singaporean',
 'restaurant_name': '\n"Straits Flavors"',
 'menu_items': '\n\nSambal Shrimp, Roti Canai, Nasi Lemak, Beef Rendang, Satay Skewers, Hainanese Chicken Rice, Char Kway Teow, Laksa Soup, Rojak Salad, Curry Puffs'}

## Agents

In [20]:
!pip install -q wikipedia
!pip install -q numexpr # for llm-math

In [21]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

In [25]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [26]:
agent.run("When was Elon Musk born? What is his age right now in 2023?")

'Elon Musk was born on June 28, 1971 and will be 52 years old in 2023.'

In [27]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent.run("When was Elon Musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 First, I need to find out when Elon Musk was born
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world, with an estimated net worth of US$213 billion as of February 2024, according to the Bloomberg Billionaires Index, and $210 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadi

'Elon Musk will be 52 years old in 2023.'